## ***1. PARÁMETROS***

In [ ]:
# General
Longitud = 50 # m
Paso = 0.5 # m
Px = [i for i in range(0, Longitud, Paso)] # m
Px.append(Longitud) # Añadir el último punto
# Carga puntual
CPuntual = [10, 5] # Ton
# Carga Distribuida
CDistribuida = [4, 8, 12] # Ton/m

## ***2. FUNCIONES PARA CARGA PUNTUAL***

In [3]:
# Calculo de reacciones por carga puntual
def ReaccionPuntual(Long, CP):
    # Reacciones en los extremos de la viga
    R1 = CP[0] * (Long - CP[1]) / Long
    R2 = CP[0] * CP[1] / Long
    return R1, R2

In [4]:
# Calculo del diagrama de momentos flectores para una carga puntual
def DMFPuntual(Long, CP, Paso=0.1):
    # Inicializa el diagrama
    DMF = []
    if CP[0] == 0:
        DMF = [0] * (int(Long / Paso) + 1)
        return DMF
    else:
        R1, R2 = ReaccionPuntual(Long, CP)
        # Calcula el momento flector en cada punto
        i = 0
        while i <= Long:
            if i < CP[1]:
                M = R1 * i
            else:
                M = R1 * i - CP[0] * (i - CP[1])
            DMF.append(M)
            i += Paso
        return DMF

In [5]:
# Calculo del diagrama de fuerzas cortantes para una carga puntual
def DFCPuntual(Long, CP, Paso=0.1):
    # Inicializa el diagrama
    DFC = []
    if CP[0] == 0:
        DFC = [0] * (int(Long / Paso) + 1)
        return DFC
    else:
        R1, R2 = ReaccionPuntual(Long, CP)
        # Calcula la fuerza cortante en cada punto
        i = 0
        while i <= Long:
            if i < CP[1]:
                F = R1
            else:
                F = R1 - CP[0]
            DFC.append(F)
            i += Paso
        return DFC

## ***3. FUNCIONES PARA CARGA DISTRIBUIDA***

In [6]:
# Cálculo de reacciones por carga distribuida
def ReaccionDistribuida(Long, CD):
    # Reacciones en los extremos de la viga
    C = CD[0] * (Long - CD[1] - CD[2])
    X = CD[1] + (Long - CD[1] - CD[2])/2
    R1 = C * (Long - X) / Long
    R2 = C * X / Long
    return R1, R2

In [7]:
# Calculo del diagrama de momentos flectores para una carga distribuida
def DMFDistribuida(Long, CD, Paso=0.1):
    # Inicializa el diagrama
    DMF = []
    if CD[0] == 0:
        DMF = [0] * (int(Long / Paso) + 1)
        return DMF
    else:
        R1, R2 = ReaccionDistribuida(Long, CD)
        # Calcula el momento flector en cada punto
        i = 0
        while i <= Long:
            if i < CD[1]:
                M = R1 * i
            elif i < Long - CD[2]:
                M = R1 * i - CD[0] * (i - CD[1]) * (Long - i + (i - CD[1])/2)
            else:
                M = R1 * i - CD[0] * (Long - CD[1] - CD[2]) * (2*i + CD[2] - CD[1] - Long)/2
            DMF.append(M)
            i += Paso
        return DMF

In [8]:
# Calculo del diagrama de fuerzas cortantes para una carga puntual
def DFCDistribuida(Long, CD, Paso=0.1):
    # Inicializa el diagrama
    DFC = []
    if CD[0] == 0:
        DFC = [0] * (int(Long / Paso) + 1)
        return DFC
    else:
        R1, R2 = ReaccionDistribuida(Long, CD)
        # Calcula la fuerza cortante en cada punto
        i = 0
        while i <= Long:
            if i < CD[1]:
                F = R1
            elif i < Long - CD[2]:
                F = R1 - CD[0] * (i - CD[1])
            else:
                F = R1 - CD[0] * (Long - CD[1]- CD[2])
            DFC.append(F)
            i += Paso
        return DFC

## ***4. FUNCIÓN PARA LA GRÁFICA***

In [ ]:
# Para la prueba
import numpy as np
long = 20
pas = 0.5
cd = [4, 0, 0]
DMF = DMFDistribuida(long, cd, pas)
DFC = DFCDistribuida(long, cd, pas)
x = np.linspace(0, long, int(long / pas) + 1)

In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

def Grafica(long, posicion, momento, cortante):
    # Crear la figura con subgráficos (2 filas, 1 columna)
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.2, 
                        subplot_titles=('Diagrama de Momento Flector (DMF)', 'Diagrama de Fuerza Cortante (DFC)'))

    # Colores modo dark
    bg_color = "#1e1e1e"  # Fondo oscuro
    grid_color = "#444"    # Color de la cuadrícula
    text_color = "#ffffff" # Color del texto
    
    # Gráfica del Momento Flector (DMF)
    fig.add_trace(go.Scatter(x=posicion, y=momento, mode='lines', fill='tozeroy',
                             line=dict(color='#00bfff'), name='Momento negativo'), row=1, col=1)
    fig.add_trace(go.Scatter(x=posicion, y=momento, mode='lines', fill='tozeroy',
                             line=dict(color='#ff4500'), name='Momento positivo'), row=1, col=1)
    fig.add_trace(go.Scatter(x=[0, long], y=[0, 0], mode='lines', 
                             line=dict(color='#800080', width=8), name='Viga'), row=1, col=1)
    
    # Gráfica de la Fuerza Cortante (DFC)
    fig.add_trace(go.Scatter(x=posicion, y=cortante, mode='lines', fill='tozeroy',
                             line=dict(color='#00bfff'), name='Cortante negativo'), row=2, col=1)
    fig.add_trace(go.Scatter(x=posicion, y=cortante, mode='lines', fill='tozeroy',
                             line=dict(color='#ff4500'), name='Cortante positivo'), row=2, col=1)
    fig.add_trace(go.Scatter(x=[0, long], y=[0, 0], mode='lines', 
                             line=dict(color='#800080', width=8), name='Viga'), row=2, col=1)
    
    # Configurar el diseño del gráfico
    fig.update_layout(height=600, width=800, showlegend=False,
                      xaxis_title='Longitud de la viga (m)',
                      xaxis2_title='Longitud de la viga (m)',
                      yaxis_title='Momento flector (Ton·m)',
                      yaxis2_title='Fuerza cortante (Ton)',
                      paper_bgcolor=bg_color,
                      plot_bgcolor=bg_color,
                      font=dict(color=text_color),
                      xaxis=dict(gridcolor=grid_color, zerolinecolor=grid_color),
                      xaxis2=dict(gridcolor=grid_color, zerolinecolor=grid_color),
                      yaxis=dict(gridcolor=grid_color, zerolinecolor=grid_color),
                      yaxis2=dict(gridcolor=grid_color, zerolinecolor=grid_color))
    
    # Mostrar el gráfico
    fig.show()

In [ ]:
Grafica(long, x, DMF, DFC)

## ***5. FUNCIÓN PARA LAS FUERZAS GENERALES***

In [10]:
# Funcion para calcular el diagrama de momentos y fuerzas cortantes globales
def Global(long, pas, esfuerzos):
    # Sumar los momentos y fuerzas por cada carga
    DMF = np.zeros(int(long / pas) + 1)
    DFC = np.zeros(int(long / pas) + 1)
    for mf in esfuerzos:
        DMF += mf[0]
        DFC += mf[1]
    return DMF, DFC

### ***5.1. EJEMPLO DE USO***

In [11]:
# Definicion de datos
Long = 20 # Longitud de la viga
pas = 0.5 # Paso de la discretización
# Definir las cargas
cp1 = [10, 5] # Carga puntual 1
cp2 = [0, 10] # Carga puntual 2
# Definir las cargas distribuidas
cd1 = [4, 0, 0] # Carga distribuida 1
cd2 = [0, 0, 8] # Carga distribuida 2

In [20]:
# Calcular los DMF y DFC por cada carga
dmf1 = DMFPuntual(Long, cp1, pas)
dfc1 = DFCPuntual(Long, cp1, pas)
dmf2 = DMFPuntual(Long, cp2, pas)
dfc2 = DFCPuntual(Long, cp2, pas)
dmf3 = DMFDistribuida(Long, cd1, pas)
dfc3 = DFCDistribuida(Long, cd1, pas)
dmf4 = DMFDistribuida(Long, cd2, pas)
dfc4 = DFCDistribuida(Long, cd2, pas)
# Calcular los diagramas globales
dmf, dfc = Global(Long, pas, [[dmf1, dfc1], [dmf2, dfc2], [dmf3, dfc3], [dmf4, dfc4]])
# Crear data frame para almacenar los resultados
import pandas as pd
import numpy as np
x = np.linspace(0, Long, int(Long / pas) + 1)
df = pd.DataFrame({'x': x, 'DMF1': dmf1, 'DFC1': dfc1, 'DMF2': dmf2, 'DFC2': dfc2, 'DMF3': dmf3, 'DFC3': dfc3, 'DMF4': dmf4, 'DFC4': dfc4, 'DMF-Global': dmf, 'DFC-Global': dfc})
df

,x,DMF1,DFC1,DMF2,DFC2,DMF3,DFC3,DMF4,DFC4,DMF-Global,DFC-Global
0,0.0,0.00,7.5,0,0,0.0,40.0,0,0,0.00,47.5
1,0.5,3.75,7.5,0,0,-19.5,38.0,0,0,-15.75,45.5
2,1.0,7.50,7.5,0,0,-38.0,36.0,0,0,-30.50,43.5
3,1.5,11.25,7.5,0,0,-55.5,34.0,0,0,-44.25,41.5
4,2.0,15.00,7.5,0,0,-72.0,32.0,0,0,-57.00,39.5
5,2.5,18.75,7.5,0,0,-87.5,30.0,0,0,-68.75,37.5
6,3.0,22.50,7.5,0,0,-102.0,28.0,0,0,-79.50,35.5
7,3.5,26.25,7.5,0,0,-115.5,26.0,0,0,-89.25,33.5
8,4.0,30.00,7.5,0,0,-128.0,24.0,0,0,-98.00,31.5
9,4.5,33.75,7.5,0,0,-139.5,22.0,0,0,-105.75,29.5


In [14]:
# Grafica de la carga puntual 1
Grafica(Long, x, dmf1, dfc1)

In [15]:
# Grafica de la carga puntual 2
Grafica(Long, x, dmf2, dfc2)

In [16]:
# Grafica de la carga Distribuida 1
Grafica(Long, x, dmf3, dfc3)

In [17]:
# Grafica de la carga Distribuida 1
Grafica(Long, x, dmf4, dfc4)

In [21]:
# Grafica global (suma de todas las cargas)
#dmf, dfc = Global(Long, pas, [[dmf1, dfc1], [dmf2, dfc2], [dmf3, dfc3], [dmf4, dfc4]])
Grafica(Long, x, dmf, dfc)